In [11]:
import pandas as pd
df = pd.read_excel('df2_data.xlsx')
df.to_csv('df2_data.csv', index=False)


In [12]:
import pandas as pd
df = pd.read_csv('df2_data.csv')
df.head()


,Age at presentation (in yrs),Age at last follow up,Sex (m/f),Religion,Max education attained,Rural/Urban,Distance from LGBRIMH (in KM),Socioeconomic status,Age at onset(in years),Time period between onset to first consultation at LGBRIMH (DUI) (in days),...,anxious,hallucination,depressed,seizure,adhd,depression1,School Adjustment,"If yes, after how many days from first presentation diagnosis changed (in days)",Continued medication 1/stopped/changed,depression2
0,8.0,10.0,0,0.000000,0.0,0.5,62.0,0.00,8.0,150,...,0,0,0,1,0,0,0.00,0,4,0
1,17.0,18.0,0,0.000000,0.4,0.5,55.0,0.00,17.0,3,...,0,0,0,1,0,0,0.66,0,4,0
2,7.0,17.0,1,0.000000,0.0,0.5,102.0,0.00,5.5,0,...,0,0,0,1,0,0,0.00,0,3,0
3,10.0,10.0,1,0.333333,0.4,0.5,29.0,0.33,0.0,10,...,0,0,0,0,1,0,1.00,0,4,0
4,8.0,15.0,0,0.000000,0.0,0.5,102.0,0.33,5.5,912,...,0,0,0,1,0,0,0.00,0,3,0


In [13]:
df4 = df

In [14]:
import pandas as pd
sorted_df = pd.read_excel('rank2.xlsx')
selected_features = sorted_df['Column Name'].tolist()[:30]
df4 = df4[selected_features]


In [15]:
df4['Final'] = df['Final']

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9088\4197094083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Final'] = df['Final']


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
import shap

X = df4.drop('Final', axis=1)
y = df4['Final']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=80, random_state=42)
model.fit(X_train, y_train)

explainer = shap.Explainer(model, X_train)
shap_values = explainer.shap_values(X_test)

results_shap = []

for i in range(len(X_test)):
    shap_instance = shap_values[i]
    
    feature_importance = pd.DataFrame(abs(shap_instance).mean(axis=0), index=X_train.columns, columns=['Weight'])
    feature_importance.reset_index(inplace=True)
    feature_importance.rename(columns={'index': 'Feature'}, inplace=True)
    
    results_shap.append(feature_importance)
    print('Instance:', i)
    print('Original prediction probabilities:', model.predict_proba(X_test.iloc[[i]])[0])
    print('SHAP explanation for the prediction:')
    print(feature_importance)
    print('\n')

df_results_shap = pd.concat(results_shap)
final_table_shap = df_results_shap.groupby('Feature').agg({'Weight': 'mean'}).reset_index()
final_table_shap['Rank'] = final_table_shap['Weight'].rank(ascending=False)

print('Final Table (SHAP):')
print(final_table_shap)


ExplainerError: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. This check failed because for one of the samples the sum of the SHAP values was 0.450000, while the model output was 0.437500. If this difference is acceptable you can set check_additivity=False to disable this check.

In [8]:
model = RandomForestClassifier(n_estimators=80, random_state=42)
model.fit(X_train, y_train)

explainer = shap.Explainer(model, X_train, check_additivity=False)  # Disable additivity check
shap_values = explainer.shap_values(X_test)


TypeError: TreeExplainer.__init__() got an unexpected keyword argument 'check_additivity'

In [16]:
import pandas as pd
import shap
from sklearn.ensemble import RandomForestClassifier

features = df4.drop(columns=['Final'])  # Features (independent variables)
target = df4['Final']  # Target variable

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(features, target)

explainer = shap.Explainer(rf_classifier, features)

shap_values = explainer.shap_values(features)
feature_importance_per_class = {class_label: {} for class_label in target.unique()}

for class_label in target.unique():
    class_indices = target[target == class_label].index
    for idx in class_indices:
        instance_shap_values = shap_values[idx]
        for feature, weight in zip(features.columns, instance_shap_values):
            if feature not in feature_importance_per_class[class_label]:
                feature_importance_per_class[class_label][feature] = []
            feature_importance_per_class[class_label][feature].append(weight)

average_feature_importance_per_class = {class_label: {feature: sum(weights) / len(weights) for feature, weights in importance.items()} for class_label, importance in feature_importance_per_class.items()}

top_features_per_class = {class_label: sorted(importance.keys(), key=lambda x: abs(importance[x]).mean(), reverse=True)[:10] for class_label, importance in average_feature_importance_per_class.items()}

for class_label, top_features in top_features_per_class.items():
    print(f"Top 10 features for class {class_label}: {top_features}")


ExplainerError: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. This check failed because for one of the samples the sum of the SHAP values was 0.090100, while the model output was 0.100000. If this difference is acceptable you can set check_additivity=False to disable this check.